In [156]:
import psycopg2
from psycopg2 import connect
from dotenv import load_dotenv
from os import environ
import pandas as pd
import textblob
from textblob import TextBlob
import plotly.graph_objects as go
import numpy as np
import streamlit
import json
from requests import get

from plotly.offline import init_notebook_mode, iplot

In [32]:
def get_db_connection():
    """Returns a database connection."""
    load_dotenv()
    return psycopg2.connect(
        host=environ["DB_HOST"],
        port=environ["DB_PORT"],
        database=environ["DB_NAME"],
        user=environ["DB_USER"],
        password=environ["DB_PASSWORD"]
        )


In [67]:
# load the comment data from the most recent set of records

# Index(['record_id', 'story_id', 'score', 'comments', 'record_time', 'story_id',
#        'title', 'author', 'story_url', 'creation_date', 'topic_id'],
#       dtype='object')

def load_stories_data() -> pd.DataFrame:
    """Loads stories with greatest score change over last 24hrs from RDS.
    Returns them as a Dataframe object."""
    query = """
        SELECT stories.story_id, 
                stories.title, 
                stories.author,
                stories.story_url,
                records.comments, 
                records.record_id, 
                stories.topic_id,
                records.score
        FROM records
        LEFT JOIN stories ON records.story_id = stories.story_id
        WHERE record_time >= NOW() - INTERVAL '1 hours'
        ;
        """
    return pd.read_sql(query, con=get_db_connection())

In [153]:
load_dotenv()
df = load_stories_data()

init_notebook_mode(connected=True)     


/var/folders/2f/000yp3dd3t359b791hdtl7jm0000gp/T/ipykernel_79900/3557222726.py:5: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



## Retrieve comment string


In [95]:
BASE_URL = "https://hacker-news.firebaseio.com/v0/item/"

def get_comment_ids(story: int) -> list:
    """Returns the id of comments from given story on Hacker News."""
    story_info = get(BASE_URL + f"{story}.json", timeout=30).json()
    comment_ids = story_info.get("kids")
    return comment_ids


def get_comments(comment: int) -> str:
    """Returns the text from a given comment."""

    comment_info = get(BASE_URL + f"{comment}.json", timeout=30).json()
    comment = comment_info.get("text")
    return comment

In [134]:
def get_comment_list(story: int):
    comments = get_comment_ids(story)
    comment_list = []
    for comment in comments:
        comment_list.append(get_comments(comment))
    return comment_list



### Sentiment analysis for each comment

In [135]:
def get_story_sentiment(story: int):
    comment_list = get_comment_list(story)
    sentiment_list = []
    for comment in comment_list:
        blob = TextBlob(str(comment))
        blob.tags           
        blob.noun_phrases   
        for sentence in blob.sentences:
            sentiment = sentence.sentiment.polarity
        sentiment_list.append({'comment': comment,
        'sentiment': sentiment})
    sentiment_mean = []
    for i in range(0,20):
        sentiment_mean.append(sentiment_list[i]['sentiment'])
    average = sum(sentiment_mean)/len(sentiment_mean)
    return average
average

-0.05570238095238096

In [147]:

value = get_story_sentiment(38865518)

if -1 <= value < -0.5:
    color="#FF0000"
elif -0.49 <= value < 0:
    color="#FFA500"
elif 0.1 <= value < 1:
    color="#FFA500"

fig = go.Figure(go.Indicator(
    mode="gauge+number",
    value=value,
    title={'text': "Overall Sentiment"},
    gauge={'axis': {'range': [-1, 1]},
        'bar': {'color': color},
        'labels': ['VERY LOW', 'LOW', 'MEDIUM', 'HIGH']
        }
))
# fig.update_layout(height=350) 

# st.plotly_chart(fig, use_container_width=True)
fig

ValueError: Invalid property specified for object of type plotly.graph_objs.indicator.Gauge: 'labels'

Did you mean "axis"?

    Valid properties:
        axis
            :class:`plotly.graph_objects.indicator.gauge.Axis`
            instance or dict with compatible properties
        bar
            Set the appearance of the gauge's value
        bgcolor
            Sets the gauge background color.
        bordercolor
            Sets the color of the border enclosing the gauge.
        borderwidth
            Sets the width (in px) of the border enclosing the
            gauge.
        shape
            Set the shape of the gauge
        steps
            A tuple of
            :class:`plotly.graph_objects.indicator.gauge.Step`
            instances or dicts with compatible properties
        stepdefaults
            When used in a template (as
            layout.template.data.indicator.gauge.stepdefaults),
            sets the default property values to use for elements of
        

In [160]:
value = get_story_sentiment(38865518)
value = -0.8

# Determine the category and corresponding color
if -1 <= value < -0.6:
    category_color = "#FF0000"
    category_label = "Very Negative"
elif -0.6 <= value < -0.3:
    category_color = "#FFA500"
    category_label = "Negative"
elif -0.3 <= value < 0.3:
    category_color = "#FFFF00"
    category_label = "Neutral"
elif 0.3 <= value < 0.6:
    category_color = "#8BFF00"
    category_label = "Positive"
else:
    category_color = "#00FF00"
    category_label = "Very Positive"

fig = go.Figure(go.Indicator(
    mode="gauge+number",
    value=value,
    title={'text': f"Overall Sentiment: {category_label}"},
    gauge={'axis': {'range': [-1, 1]},
           'bar': {'color': category_color},
           }
))

fig.update_layout(
    xaxis=dict(
        tickmode='linear',
        tickvals=[-1,-0.5,0,0.5,1],
        ticktext=["😡", "😠", "😐", "😊", "😃"]
    )
)
fig
# iplot(fig)

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'gauge': {'axis': {'range': [-1, 1]}, 'bar': {'color': '#FF0000'}},
              'mode': 'gauge+number',
              'title': {'text': 'Overall Sentiment: Very Negative'},
              'type': 'indicator',
              'value': -0.8}],
    'layout': {'template': '...',
               'xaxis': {'tickmode': 'linear', 'ticktext': ['😡', '😠', '😐', '😊', '😃'], 'tickvals': [-1, -0.5, 0, 0.5, 1]}}
})

In [144]:
value = get_story_sentiment(38865518)

# Determine the category and corresponding color and emoji
if -1 <= value < -0.6:
    category_label = "😡 Very Negative"
    category_color = "#FF0000"
elif -0.6 <= value < -0.3:
    category_label = "😠 Negative"
    category_color = "#FFA500"
elif -0.3 <= value < 0.3:
    category_label = "😐 Neutral"
    category_color = "#FFFF00"
elif 0.3 <= value < 0.6:
    category_label = "😊 Positive"
    category_color = "#8BFF00"
else:
    category_label = "😃 Very Positive"
    category_color = "#00FF00"

fig = go.Figure(go.Indicator(
    mode="gauge+number",
    value=value,
    title={'text': f"Overall Sentiment {category_label}"},
    gauge={'axis': {'range': [-1, 1]},
           'bar': {'color': category_color},
           }
))

fig

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'gauge': {'axis': {'range': [-1, 1]}, 'bar': {'color': '#FFFF00'}},
              'mode': 'gauge+number',
              'title': {'text': 'Overall Sentiment 😐 Neutral'},
              'type': 'indicator',
              'value': -0.05570238095238096}],
    'layout': {'template': '...'}
})

In [173]:
import plotly.graph_objects as go

# Example value
value = get_story_sentiment(38865518)

# Determine the category and corresponding color and emoji
if -1 <= value < -0.6:
    category_label = "😡 Very Negative"
    category_color = "#FF0000"
elif -0.6 <= value < -0.3:
    category_label = "😠 Negative"
    category_color = "#FFA500"
elif -0.3 <= value < 0.3:
    category_label = "😐 Neutral"
    category_color = "#FFFF00"
elif 0.3 <= value < 0.6:
    category_label = "😊 Positive"
    category_color = "#8BFF00"
else:
    category_label = "😃 Very Positive"
    category_color = "#00FF00"

fig = go.Figure(go.Indicator(
    mode="gauge+number",
    value=value,
    title={'text': f"Overall Sentiment {category_label}"},
    gauge={'axis': {'range': [-1, 1]},
           'bar': {'color': category_color},
           }
))

fig


ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'gauge': {'axis': {'range': [-1, 1]}, 'bar': {'color': '#FFFF00'}},
              'mode': 'gauge+number',
              'title': {'text': 'Overall Sentiment 😐 Neutral'},
              'type': 'indicator',
              'value': -0.05570238095238096}],
    'layout': {'template': '...'}
})

In [17]:
text = '''
I am very very happy.
'''

blob = TextBlob(text)
blob.tags           

blob.noun_phrases   
for sentence in blob.sentences:
    print(sentence.sentiment.polarity)

1.0


## Produce speedometer by topic